In [ ]:
import numpy as np
import pandas as pd
import collections
import scipy.cluster.hierarchy as hcl
from itertools import combinations

from scipy.spatial.distance import squareform

mbti_score = np.matrix([
    [4, 4, 4, 5, 4, 5, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1],
    [4, 4, 5, 4, 5, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1],
    [4, 5, 4, 4, 4, 4, 4, 5, 1, 1, 1, 1, 1, 1, 1, 1],
    [5, 4, 4, 4, 4, 4, 4, 4, 5, 1, 1, 1, 1, 1, 1, 1],
    [4, 5, 4, 4, 4, 4, 4, 5, 3, 3, 3, 3, 2, 2, 2, 2],
    [5, 4, 4, 4, 4, 4, 5, 4, 3, 3, 3, 3, 3, 3, 3, 3],
    [4, 4, 4, 4, 4, 5, 4, 4, 3, 3, 3, 3, 2, 2, 2, 5],
    [4, 4, 5, 4, 5, 4, 4, 4, 3, 3, 3, 3, 2, 2, 2, 2],
    [1, 1, 1, 5, 3, 3, 3, 3, 2, 2, 2, 2, 3, 5, 3, 5],
    [1, 1, 1, 1, 3, 3, 3, 3, 2, 2, 2, 2, 5, 3, 5, 3],
    [1, 1, 1, 1, 3, 3, 3, 3, 2, 2, 2, 2, 3, 5, 3, 5],
    [1, 1, 1, 1, 3, 3, 3, 3, 2, 2, 2, 2, 5, 3, 5, 3],
    [1, 1, 1, 1, 2, 3, 2, 2, 3, 5, 3, 5, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 3, 2, 2, 5, 3, 5, 3, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 3, 2, 2, 3, 5, 3, 5, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 3, 5, 2, 5, 3, 5, 3, 4, 4, 4, 4]
])

MBTI_MAP = dict(
    INFP=0,
    ENFP=1,
    INFJ=2,
    ENFJ=3,
    INTJ=4,
    ENTJ=5,
    INTP=6,
    ENTP=7,
    ISFP=8,
    ESFP=9,
    ISTP=10,
    ESTP=11,
    ISFJ=12,
    ESFJ=13,
    ISTJ=14,
    ESTJ=15
)


class MbtiResult:
    def __init__(self, member_mbti_dict):
        self._mbti_dict = member_mbti_dict.copy()
        self._member_num = [k for k in self._mbti_dict.keys()]
        self._member_vibe = collections.defaultdict(dict)
        self._member_score = collections.defaultdict(int)
        self._mbti_inverted = collections.defaultdict(list)
        self._linkage = None

        self._init_mbti_inverted()
        self._init_vibe()

    def _init_mbti_inverted(self):
        for key, value in self._mbti_dict.items():
            self._mbti_inverted[value].append(key)

        self._mbti_inverted = [(k, v) for k, v in self._mbti_inverted.items()]
        self._mbti_inverted.sort(key=lambda tup: len(tup[1]), reverse=True)


    def _init_vibe(self):
        for i in range(0, len(self._member_num)):
            for j in range(0, len(self._member_num)):
                first_member, first_member_mbti = self._get_member_mbti(i)
                second_member, second_member_mbti = self._get_member_mbti(j)
                goodness = mbti_score.item((MBTI_MAP[first_member_mbti], MBTI_MAP[second_member_mbti]))
                self._member_vibe[first_member][second_member] = goodness


    def _get_member_mbti(self, idx):
        return (self._member_num[idx], self._mbti_dict[self._member_num[idx]])


    @property
    def mbti_inverted(self):
        return self._mbti_inverted

    @property
    def mbti_distance_matrix(self):
        data = {}
        idx = 0
        for member, vibe in self._member_vibe.items():
            data[idx] = [5 - goodness for _, goodness in vibe.items()]
            data[idx][idx] = 0
            idx += 1

        member_vibe = pd.DataFrame(data)
        return member_vibe

    @property
    def mbti_dict(self):
        return self._member_vibe

    @property
    def member(self):
        return self._member_num

    # def make_dic(self):
    #   for member, vibe in self.__member_vibe.items():
    #     #member dictionary에 저장
    #         for another_member, goodness in vibe.items():
              #여기서 another_member, goodness for문 돌리면서 dictionary에 list로 저장

    def print_vibe(self):
        for member, vibe in self._member_vibe.items():
            for another_member, goodness in vibe.items():
                print(f'{member}({self._mbti_dict[member]})님 + {another_member}({self._mbti_dict[another_member]})님 = {self._member_vibe[member][another_member]}')

In [ ]:
part_mbti = {
    '_belle.mxmxnt': 'ENTJ',
    'wasabihater': 'ESFJ',
    'yumi': 'ESFJ',
    'junseo': 'ESTP',
    'bottle': 'ENFP'
}

analysis = MbtiResult(part_mbti)

print('MBTI 궁합')
analysis.print_vibe()


MBTI 궁합
_belle.mxmxnt(ENTJ)님 + _belle.mxmxnt(ENTJ)님 = 4
_belle.mxmxnt(ENTJ)님 + wasabihater(ESFJ)님 = 3
_belle.mxmxnt(ENTJ)님 + yumi(ESFJ)님 = 3
_belle.mxmxnt(ENTJ)님 + junseo(ESTP)님 = 3
_belle.mxmxnt(ENTJ)님 + bottle(ENFP)님 = 4
wasabihater(ESFJ)님 + _belle.mxmxnt(ENTJ)님 = 3
wasabihater(ESFJ)님 + wasabihater(ESFJ)님 = 4
wasabihater(ESFJ)님 + yumi(ESFJ)님 = 4
wasabihater(ESFJ)님 + junseo(ESTP)님 = 3
wasabihater(ESFJ)님 + bottle(ENFP)님 = 1
yumi(ESFJ)님 + _belle.mxmxnt(ENTJ)님 = 3
yumi(ESFJ)님 + wasabihater(ESFJ)님 = 4
yumi(ESFJ)님 + yumi(ESFJ)님 = 4
yumi(ESFJ)님 + junseo(ESTP)님 = 3
yumi(ESFJ)님 + bottle(ENFP)님 = 1
junseo(ESTP)님 + _belle.mxmxnt(ENTJ)님 = 3
junseo(ESTP)님 + wasabihater(ESFJ)님 = 3
junseo(ESTP)님 + yumi(ESFJ)님 = 3
junseo(ESTP)님 + junseo(ESTP)님 = 2
junseo(ESTP)님 + bottle(ENFP)님 = 1
bottle(ENFP)님 + _belle.mxmxnt(ENTJ)님 = 4
bottle(ENFP)님 + wasabihater(ESFJ)님 = 1
bottle(ENFP)님 + yumi(ESFJ)님 = 1
bottle(ENFP)님 + junseo(ESTP)님 = 1
bottle(ENFP)님 + bottle(ENFP)님 = 4
